## Loading Data

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers
import tensorflow_addons as tfa
from tensorflow import data as tf_data

In [2]:
train_60,val_20,test_20=tfds.load("oxford_iiit_pet",split=["train+test[:20%]","test[20%:60%]","test[60%:]"],as_supervised=True)

resizing=layers.Resizing(224,224)
train_60=train_60.map(lambda x,y:(resizing(x),y))
val_20=val_20.map(lambda x,y:(resizing(x),y))
test_20=test_20.map(lambda x,y:(resizing(x),y))

batch_size = 64

train_60 = train_60.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
val_20 = val_20.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test_20 = test_20.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()

In [3]:
train,test=tfds.load("oxford_iiit_pet",split=["train","test"],as_supervised=True)

resizing=layers.Resizing(224,224)
train=train.map(lambda x,y:(resizing(x),y))
test=test.map(lambda x,y:(resizing(x),y))

batch_size = 64

train = train.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test = test.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()

In [ ]:
F10_train=tfds.load("oxford_iiit_pet",split=[f"train[:{r}%]+train[{r+10}%:]" for r in range(0,100,10)],as_supervised=True)
F10_val=tfds.load("oxford_iiit_pet",split=[f"train[{r}%:{r+10}%]" for r in range(0,100,10)],as_supervised=True)

## Model

In [3]:
base_model=keras.applications.ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3),pooling="avg")

In [4]:
base_model.trainable=False

In [10]:
inputs=keras.Input(shape=(224,224,3))   
preprocessing=tf.keras.applications.resnet.preprocess_input(inputs)
x=base_model(preprocessing, training=False)
x=layers.Dense(256,activation="relu")(x)
x=layers.Dropout(0.5)(x)    
outputs=layers.Dense(37)(x)

model=keras.Model(inputs,outputs)

loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizers=keras.optimizers.Adam(learning_rate=0.0001,decay=0.00001)

model.compile(optimizer=optimizers,loss=loss_fn,metrics=['accuracy'])

In [11]:
model.summary(show_trainable=True) 

Model: "model_2"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 tf.__operators__.getitem_2   (None, 224, 224, 3)      0         Y          
 (SlicingOpLambda)                                                          
                                                                            
 tf.nn.bias_add_2 (TFOpLambd  (None, 224, 224, 3)      0         Y          
 a)                                                                         
                                                                            
 resnet50 (Functional)       (None, 2048)              23587712  N          
                                                                            
 dense_4 (Dense)             (None, 256)               5245

In [13]:
model.fit(train_60,epochs=10,validation_data=val_20)

Epoch 1/10


69/69 [==============================] - 17s 240ms/step - loss: 0.3797 - accuracy: 0.8913 - val_loss: 0.4106 - val_accuracy: 0.8773
Epoch 2/10
69/69 [==============================] - 16s 240ms/step - loss: 0.3461 - accuracy: 0.9017 - val_loss: 0.4062 - val_accuracy: 0.8848
Epoch 3/10
69/69 [==============================] - 17s 240ms/step - loss: 0.3196 - accuracy: 0.9085 - val_loss: 0.3865 - val_accuracy: 0.8807
Epoch 4/10
69/69 [==============================] - 17s 243ms/step - loss: 0.2912 - accuracy: 0.9166 - val_loss: 0.3786 - val_accuracy: 0.8793
Epoch 5/10
69/69 [==============================] - 17s 242ms/step - loss: 0.2700 - accuracy: 0.9243 - val_loss: 0.3715 - val_accuracy: 0.8780
Epoch 6/10
69/69 [==============================] - 17s 243ms/step - loss: 0.2705 - accuracy: 0.9187 - val_loss: 0.3731 - val_accuracy: 0.8834
Epoch 7/10
69/69 [==============================] - 17s 241ms/step - loss: 0.2356 - accuracy: 0.9334 - val_loss: 0.3628 - val_accuracy: 0.8862
Epoch 8/10

In [14]:
model.evaluate(test_20)

23/23 [==============================] - 4s 181ms/step - loss: 0.2848 - accuracy: 0.9142


[0.2848379909992218, 0.9141689538955688]